# Machine Learning workflow

by Anaïs Pepey

🦟 In this notebook, we will import open data and apply basic machine learning concepts to analyse the relationship between different variables related to malaria. 

🗣️ Please call Asia or Anaïs if you feel stuck or have any question!

🐍 Happy coding!

## Import libraries

### all-time basics

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

### machine learning specifics

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score

## 0. Data Exploration and Visualisation

### Understanding our data

🌏 Let's have a look at this data provided by the [World Bank Open Data Page](https://data.worldbank.org/) for the year 2020.

In [ ]:
df = pd.read_csv("./data/workshop/worldbank-df.csv")


👇 Display the first five rows of the dataframe `df`.

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
df.head()
```

</details>

🎲 We can also look at random rows, instead of the first 5:

In [ ]:
df.sample(5)

If you re-run the cell, you will obtain 5 different rows. Try it!

Python simplifies our work by providing core functions.

👇 Run the function below to have a look at the shape of the dataframe, which will return its `(number of rows, number of columns)`.

In [ ]:
df.shape

The dataframe `df` includes 214 rows and 13 columns. It is small by machine learning standards!

👇 We can also peek at the first 5 rows:

In [ ]:
df.head()

👇 Run the function below to get a quick glance at the distribution of numerical variables accross the whole dataset. 

In [ ]:
df.describe()

👇 Run the function below to do the same for a specific column. 

In [ ]:
df["incidence_per_1000"].describe()

❓ How would you look at the details of the `life_expectancy` column?

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
df["life_expectancy"].describe()
```

</details>

❓ What is the mean value of access to electricity?

In [ ]:
mean_elec = df['access_electricity'].mean()
mean_elec

👇 We can round the number to the amount of decimal we require, for example only one decimal:

In [ ]:
n_decimal = 1

In [ ]:
round(mean_elec, n_decimal)

👇  We can create sentences that integrate variables. Take a minute to understand every element of the cell below:

In [ ]:
var1 = round(df['rural_pop'].min(), n_decimal)
var2 = round(df['rural_pop'].max(), n_decimal)

print(f"In 2020, depending on the country, between {var1} and {var2}% of the population lived in rural areas.")

❓ Can you find the maximum value of malaria incidence? Store it under the `max_malaria` variable. 

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
max_malaria = df['incidence_per_1000'].max()
max_malaria
```

</details>

👇 Another easy method to see which countries harbour the most malaria cases is to sort the dataframe:

In [ ]:
df.sort_values(by='incidence_per_1000', ascending=False)

👇 We can also choose to display the ten countries with the highest malaria incidence:

In [ ]:
df.sort_values(by='incidence_per_1000', ascending=False)[:10]

### Plotting our data

📈 A good practice is to plot the data to understand better the relationship between the different variables. 

👇 Run the function below to create a scatterplot.

In [ ]:
sns.scatterplot(x='life_expectancy', y='incidence_per_1000', data=df, alpha=0.5, hue = "continent");

📊 We can plot to relation between the two variables:

In [ ]:
sns.scatterplot(x='life_expectancy', y='incidence_per_1000', data=df, alpha=0.5);
sns.regplot(x='life_expectancy', y='incidence_per_1000', data=df, scatter=False, color = 'red')

### Plot like a pro!
📈 Let's see all the relationships between our columns:

In [ ]:
sns.pairplot(df, hue = 'continent', corner = True); #might require a few seconds to load

## 1. Data preparation

### Data cleaning

🧼 Let's save the initial shape of our dataframe to see how it evolves through cleaning. 

In [ ]:
initial_shape = df.shape
initial_shape

Before using the dataset, we need to make sure it is usable. 

👇 First, we clean all the NA (Non Applicable) values.

In [ ]:
df_test = df.dropna()
noNA_shape = df_test.shape

print(f"The dataset size went from {initial_shape} to {noNA_shape} after cleaning all NAs.")

😱 Oh, no! We lost a significant amount of data. Let's understand where those NAs are.

In [ ]:
df.isna().sum()

😌 It seems like only two columns are responsible for the most NAs. Let's delete those.

In [ ]:
cols_to_remove = ["population", "pop_growth"]

df_cleaned = df.drop(columns = cols_to_remove)
df_cleaned.head()

In [ ]:
df_cleaned.isna().sum()

🤗 Much better! Now we can delete the rows containing NAs without losing so much data.

In [ ]:
df_cleaned = df_cleaned.dropna()
df_cleaned.shape

👇 Then, we remove all duplicate values (if any).

In [ ]:
df_cleaned = df_cleaned.drop_duplicates()

In real life, cleaning data includes [a lot more steps](https://towardsdatascience.com/clean-efficient-data-pipelines-with-pythons-sklearn-2472de04c0ea) and can get tricky.

Cleaning and preprocessing data can be the longest part of the process and can be considered the most important one. 

## 2. Model training

### Choosing your model

We start with the easiest model of them all: the linear regression, imported at the beginning on the notebook from the [Scikit-Learn](https://scikit-learn.org) library. 

<details><summary markdown='span'>Additional info
</summary>

  Scikit-Learn is an open source, well documented and honestly life saving initiative for all mahcine learning enthusiasts. 
  Do not hesitate to read their [documentation](https://scikit-learn.org/stable/user_guide.html) and explore their [tutorials](https://scikit-learn.org/stable/auto_examples/index.html). 

  A linear regression is a mathematical model that can be written as f(x) = ax + b
  where a is the slope and b is the intercept. 
</details>


In [ ]:
model = LinearRegression()

👇 The model is now loaded under the `model` variable:

In [ ]:
model

### Defining the feature and the target

Let's start with a simple model.

We assume that malaria incidence can be predicted with the life expectancy of a country.

👇 We define that our feature `X` is `life_expectancy` and our target `y` is `incidence_per_1000`.

In [ ]:
X = df_cleaned[['life_expectancy']]
y = df_cleaned[['incidence_per_1000']]

plt.scatter(X, y, alpha = 0.2)

### Reserving some data for validation

⚡ This is a very important step!

To be able to properly assess the efficacy of our model, we should keep some data 'unseen'.
It means dividing the dataset into a `training` part and a `testing` part. 

Steps:
1. the model learns on some part of the dataset
2. the model predicts using the other part: it uses the feature(s) `X` to make a prediction of the target `y`. 
3. we compare the prediction and the true values

👇  We first choose the proportion of data kept for `testing`. It is usually 0.2 or 0.3, meaning 20 or 30%, selected at random. 

In [ ]:
size = 0.3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = size)


In [ ]:
X_train.head()

In [ ]:
y_train.head()

❓ What do you expect the approximative shapes of `X_test` and `y_test` will be?

👇 Print the shapes of `X_test` and `y_test` below.

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
print(X_test.shape, y_test.shape)
```

</details>

### Fitting your model

👇 We make the model learn from the `training` dataset:

In [ ]:
model_fitted = model.fit(X_train, y_train)

plt.scatter(X_train, y_train, alpha = 0.2)

# 3. Model prediction

👇 Then, we can ask the model to predict malaria incidence values `y` from the the values of life expectancy `X` it has not seen before. 

In [ ]:
prediction = model_fitted.predict(X_test)

👇 Here, we can see the predicted values calculated from `X_test` in red, against the actual values from `X_train`, `y_train` in blue.

In [ ]:
plt.scatter(X_train, y_train, alpha = .2),
plt.scatter(X_test, prediction, alpha = .2, color = 'red')

🤔 It seems that our model is not doing an excellent job...

👇 But how much of a bad prediction is that? Let's measure it!

## 4. Model evaluation

### R2 score

In [ ]:
model_score_baseline = model_fitted.score(X_test, y_test)
print("Model R2 score:", round(model_score_baseline, 2))

<details><summary markdown='span'>What is the R2 score?
</summary>

In linear regression, the R-squared score (also known as the coefficient of determination) measures the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

A higher R-squared score indicates that the model explains a larger portion of the variability in the dependent variable, suggesting a stronger relationship between the independent and dependent variables.
</details>

### Accuracy

In [ ]:
accuracy_baseline = cross_val_score(model_fitted, X_train, y_train).mean()
print("Model accuracy:", round(accuracy_baseline, 2))

<details><summary markdown='span'>What is accuracy?
</summary>

Model accuracy is defined as the percentage of correct classifications made by the model, calculated by dividing the number of correct predictions by the total number of predictions across all classes. 

This metric is expressed as a value between 0 and 1 (or 0% and 100%), where 0 indicates the model always predicts incorrectly, and 1 (or 100%) signifies that it always predicts correctly.

</details>

👇 We will now try to improve our model and increase the scores, which are currently low.

# 5. Improving the model

## Option 1: More features

### Defining the features

🔁 We can repeat this workflow, including more variables from our dataset. 
To do so, we change our features `X` to include more columns, and we keep `y` as is. 

In [ ]:
Xs = df_cleaned[['life_expectancy', 'forest_area', 'rural_pop', 'access_electricity', 'GDP_growth', 'working_age_pop']]
y = df_cleaned[['incidence_per_1000']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size = size)


In [ ]:
X_train.head()

👇 The shape of our features `X` will now be different from before:

In [ ]:
X_train.shape

👇  However, `y_train` remains the same:

In [ ]:
y_train.shape

### Fitting the model 

In [ ]:
model_fitted2 = model.fit(X_train, y_train)

### Evaluating the model

In [ ]:
model_score2 = model_fitted2.score(X_test, y_test)
print("Model R2 score:", round(model_score2, 2))

In [ ]:
accuracy2 = cross_val_score(model_fitted2, X_train, y_train).mean()
print("Model accuracy:", round(accuracy2, 2))

🙌 Our model improved!

There is still a long way to go, but you get the gist...

### Prediction

Let's compare the updated prediction to the previous one: 

🔮 First by making a new `prediction` with the `testing` data:

In [ ]:
prediction2 = model_fitted.predict(X_test)

📊 Then creating the plot by choosing the `life expectancy` variable:

In [ ]:
plt.scatter(X_train['life_expectancy'], y_train, alpha = .2),
plt.scatter(X_test['life_expectancy'], prediction2, alpha = .2, color = 'red')

How would you plot the `prediction` of the malaria incidence `y` against an other variable, such as `working_age_pop`? 

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>
plt.scatter(X_train['working_age_pop'], y_train, alpha = .2),
plt.scatter(X_test['life_expectancy'], prediction2, alpha = .2, color = 'red')
</details>

### Features importance

📊 Let's plot the importance of each feature:

First we create a dataframe that includes the name of the `features` and their respective `importance` in the `model`:

In [ ]:
features = pd.DataFrame(X_train.columns)
features.columns = ['features']

importance = pd.DataFrame(abs(model_fitted2.coef_.T))
importance.columns = ['importance']

result = pd.concat([features, importance], axis=1).sort_values(by='importance', ascending = False)

📉 Then we plot:

In [ ]:
ax = sns.barplot(x = 'features', y = 'importance', hue = 'features', data = result)
ax.tick_params(axis = 'x', rotation = 90)

❓ Which feature was the most important?

👇  Uncomment and complete the code below.

In [ ]:
# most_important_feature = ?

If you made it this far, great!

The rest of the notebook is optional, you can keep going if you have plenty of time or keep them for later (or never!), they will stay online and openly available.

### Option 2: A different model

### Ridge Regression

Let's import a different model from Sickit-Learn: the <b>Ridge</b>.

<details><summary markdown='span'>About Ridge
</summary>
The primary difference between Ridge Regression and Linear Regression in scikit-learn lies in the addition of a regularization term in Ridge Regression, which helps prevent overfitting by penalizing large coefficients.

</details>

In [ ]:
model_ridge = Ridge()
model_ridge

👇 Fit `model ridge` to `(X_train, y_train)` and save the fitted model as `model_fitted3`

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
model_fitted3 = model_ridge.fit(X_train, y_train)
```

</details>

👇 Calculate the R2 score and save the result as `model_score3`: 

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
model_score3 = model_fitted3.score(X_test, y_test)
```

</details>

👇 Calculate the model accuracy and save the result as `accuracy3`: 

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
accuracy3 = cross_val_score(model_fitted3, X_train, y_train).mean()
```

</details>

This model should be more accurate than the baseline one. Run the cell below to verify:

In [ ]:
if model_score3 > model_score_baseline and accuracy3 > accuracy_baseline:
    print("This model performs better than the baseline one. All correct!")
else:
    print("Something's not right... Have a look at your code to correct it or call Anaïs/ Asia!")

💡 Here, we expect to get a similar score and accuracy than previously obtained with `LinearRegression()`, because they are both based on the same mathematical principle.

👇 Let's define a function to help us define if two metrics are almost equal:

In [ ]:
def compare_values(value1, value2, decimal):
    try:
        np.testing.assert_almost_equal(value1, value2, decimal=decimal)
        print("Values are almost equal.")
    except AssertionError:
        print("Values are not almost equal.")

❓ How would you test if `model_score2` and `model_score3` are almost equal (by 3 decimals)? 

👇 Write your code in the cell below.

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
decimal = 3
compare_values(model_score2, model_score3, decimal)
```

</details>

❓ What about `accuracy2` and `accuracy3`? 

👇 Write your code in the cell below.

In [ ]:
# type, uncomment and run your code here

<details><summary markdown='span'>View solution
</summary>

```python
compare_values(accuracy2, accuracy3, decimal)
```

</details>

<details><summary markdown='span'>More on Regression Models
</summary>

[Other regression models](https://scikit-learn.org/stable/modules/linear_model.html) exist, such as Lasso or Elastic Net. 

The choice between all available methods should be guided by the specific characteristics of the dataset and the objectives of the analysis.

Do not hesitate to read [more](https://towardsdatascience.com/whats-the-difference-between-linear-regression-lasso-ridge-and-elasticnet-8f997c60cf29) on the topic!

</details>

### Polynomial Features

🔥 Another option is to make our linear model handle non-linear relationship... That sounds very conterintuitive but it can be done through <b>polynomial features</b>.

Let's got through the basic steps. Do not hesitate to modify the code to explore more of it!

In [ ]:
#defining the degrees
degrees = 2

In [ ]:
#sorting the values
sorted_df = df_cleaned.sort_values('life_expectancy')

#defining the feature and target
X_sorted = sorted_df[['life_expectancy']]
y_sorted = sorted_df['incidence_per_1000']

#creating our polynomial features
poly_features = PolynomialFeatures(degree=degrees, include_bias=False)
X_poly = poly_features.fit_transform(X_sorted)

#fitting model
model_poly = model.fit(X_poly, y_sorted)

#predicting
predictions = model_poly.predict(X_poly)

#plotting predictions over original data
sns.scatterplot(x=X_sorted['life_expectancy'], y=y_sorted, alpha=0.5)
plt.plot(X_sorted['life_expectancy'], predictions, linewidth=3, color='r')

In [ ]:
#scoring the model
score_poly = model_fitted.score(X_poly, y_sorted)
print("Model R2 score:", round(score_poly, 2))

### Option 3: Joining different datasets


Machine Learning is mostly about data.

By creating bigger datasets, we might (or not!) improve the scores and the capacities of our model.

In [ ]:
df_gdp = pd.read_csv("./data/workshop/GDPpercapita.csv")
df_gdp.describe()

👇 We join the datasets by matching their `country` columns: 

In [ ]:
df_complete = pd.merge(df, df_gdp, how = 'left', on = 'country')
df_complete.head()

You now have a whole new feature to test on!
Ideally, we would look for more specific data on malaria, such as mosquito net coverage, access to treatment... But it is difficult to obtain clean, standardised data at a global scale. 

### Option 4: Actual preprocessing

To make the notebook easy to follow, we skipped an important step of preprocessing: <b>Data Scaling</b>. 

You can have a look at [how you can implement it](https://betterdatascience.com/data-scaling-for-machine-learning/) and try it on your own if you feel confident. 

### Option 5: Your own solution!

Possibilities are almost endless and even if we have to adhere to good practices, your creativity can save the day!

Do not hesitate to explore the [available tools](https://scikit-learn.org/stable/index.html), to look for [new data](https://www.kaggle.com/) to analyse, or check regurlarly this [repo](https://github.com/APepey/ML-workshop) for new resources and challenges!